In [1]:
import os
import re
import pandas as pd
import numpy as np
from tqdm import tqdm

In [2]:
column_dict = {
    'city':'city',
    'yearmonth':'yearmonth',
    '鄉鎮市區':'district',
'交易標的':'trade_target',
'土地區段位置建物區段門牌':'address',
'土地移轉總面積平方公尺':'land_area',
'都市土地使用分區':'area_use',
'非都市土地使用分區':'nonmetro_d',
'非都市土地使用編定':'nonmetro_u',
'交易年月日':'trade_date',
'交易筆棟數':'target_detail',
'移轉層次':'trade_floor',
'總樓層數':'total_floor',
'建物型態':'state',
'主要用途':'purpose',
'主要建材':'materials',
'建築完成年月':'building_date',
'建物移轉總面積平方公尺':'building_area',
'建物現況格局-房':'room',
'建物現況格局-廳':'hall',
'建物現況格局-衛':'health',
'建物現況格局-隔間':'compartment',
'有無管理組織':'manage',
'總價元':'price',
'單價元平方公尺':'unit_price',
'車位類別':'berth_type',
'車位移轉總面積平方公尺':'berth_area',
'車位總價元':'berth_price',
'備註':'note',
'編號':'trade_id'}
column_dict_inv = {v: k for k, v in column_dict.items()}

In [3]:
df = pd.read_csv('../original/107_1/A_lvr_land_A.csv')
df.drop([0], inplace=True)
col = list(df.columns)
col.insert(0, 'city')
merge_df = pd.DataFrame(columns=col)
print(merge_df)

Empty DataFrame
Columns: [city, 鄉鎮市區, 交易標的, 土地區段位置建物區段門牌, 土地移轉總面積平方公尺, 都市土地使用分區, 非都市土地使用分區, 非都市土地使用編定, 交易年月日, 交易筆棟數, 移轉層次, 總樓層數, 建物型態, 主要用途, 主要建材, 建築完成年月, 建物移轉總面積平方公尺, 建物現況格局-房, 建物現況格局-廳, 建物現況格局-衛, 建物現況格局-隔間, 有無管理組織, 總價元, 單價元平方公尺, 車位類別, 車位移轉總面積平方公尺, 車位總價元, 備註, 編號]
Index: []

[0 rows x 29 columns]


In [4]:
city_dict = {
    'A':'臺北市', 'J':'新竹縣', 'S':'高雄縣', 
    'B':'臺中市', 'K':'苗栗縣', 'T':'屏東縣', 
    'C':'基隆市', 'L':'臺中縣', 'U':'花蓮縣', 
    'D':'臺南市', 'M':'南投縣', 'V':'臺東縣', 
    'E':'高雄市', 'N':'彰化縣', 'W':'金門縣', 
    'F':'臺北縣', 'O':'新竹市', 'X':'澎湖縣', 
    'G':'宜蘭縣', 'P':'雲林縣', 'Y':'陽明山', 
    'H':'桃園縣', 'Q':'嘉義縣', 'Z':'連江縣', 
    'I':'嘉義市', 'R':'臺南縣'}

In [5]:
for foldername in os.listdir("../original"):
    if re.match("10*", foldername):
        i = 0
        for filename in os.listdir("../original/" + foldername):
            if re.match("[A-Z]_lvr_land_*", filename):
                i+=1
#                 print('../original/'+ foldername + '/' + filename)
                tmp_df = pd.read_csv('../original/'+ foldername + '/' + filename)
                tmp_df.drop([0], inplace=True)
#                 print(merge_df.shape, tmp_df.shape)
                tmp_df['city'] = city_dict[filename[0]]
                merge_df = merge_df.append(tmp_df, ignore_index=True)
        print(foldername, i)

107_1 22
107_2 22
107_3 22
107_4 22
108_1 22
108_2 22
108_3 22
108_4 22
109_1 22


In [6]:
print(merge_df.shape)
merge_df = merge_df.loc[[len(x) >= 6 for x in merge_df['交易年月日']]]
merge_df.reset_index(drop=True, inplace=True)
print(merge_df.shape)
merge_df = merge_df[merge_df['鄉鎮市區'].notna()]
merge_df.reset_index(drop=True, inplace=True)
print(merge_df.shape)
merge_df['yearmonth'] = [ x[:-2] for x in merge_df['交易年月日']]
merge_df.rename(columns=column_dict, inplace=True)
merge_df = merge_df.replace(np.nan, 'NULL', regex=True)

(724394, 29)
(724393, 29)
(724391, 29)


In [7]:
# decompose
land_df = merge_df[['trade_target', 'area_use','compartment', 'manage']].copy()
land_df = land_df.drop_duplicates()
land_df.reset_index(drop=True, inplace=True)
land_df['num'] = range(len(land_df))
print(land_df)
land_df.to_csv('3_merge_1.csv', index=False, na_rep='NULL')
# print(land_df.shape)

    trade_target area_use compartment manage  num
0      房地(土地+建物)        商           有      有    0
1   房地(土地+建物)+車位        商           有      有    1
2      房地(土地+建物)        住           有      有    2
3             土地       其他           有      無    3
4      房地(土地+建物)        住           有      無    4
..           ...      ...         ...    ...  ...
83            建物        住           有      有   83
84            土地       其他           無      無   84
85            車位       其他           無      無   85
86            土地        住           無      無   86
87            土地        住           無      有   87

[88 rows x 5 columns]


In [8]:
struct_df = merge_df[['nonmetro_d', 'nonmetro_u', 'state', 'purpose', 'materials', 'hall', 'berth_type']].copy()
struct_df.drop_duplicates(inplace=True)
struct_df.reset_index(drop=True, inplace=True)
struct_df['num'] = range(len(struct_df))
struct_df.to_csv('3_merge_2.csv', index=False, na_rep='NULL')
print(struct_df)

     nonmetro_d nonmetro_u            state  purpose    materials hall  \
0          NULL       NULL  住宅大樓(11層含以上有電梯)      商業用       鋼筋混凝土造    2   
1          NULL       NULL  住宅大樓(11層含以上有電梯)  見其他登記事項     鋼骨鋼筋混凝土造    2   
2          NULL       NULL  住宅大樓(11層含以上有電梯)     國民住宅       鋼筋混凝土造    2   
3          NULL       NULL  住宅大樓(11層含以上有電梯)  見其他登記事項       鋼筋混凝土造    2   
4          NULL       NULL  住宅大樓(11層含以上有電梯)  見其他登記事項       鋼筋混凝土造    1   
...         ...        ...              ...      ...          ...  ...   
6189        風景區       農牧用地               農舍  見其他登記事項      見其他登記事項    0   
6190       NULL       NULL               其他      住商用           木造    1   
6191        風景區     丙種建築用地              透天厝      住家用           竹造    1   
6192       NULL       NULL    華廈(10層含以下有電梯)  見其他登記事項       鋼筋混凝土造   48   
6193       NULL       NULL              透天厝      住家用  鋼筋混凝土加強空心磚造    3   

     berth_type   num  
0          NULL     0  
1          坡道平面     1  
2          NULL     2  
3          NULL

In [9]:
other_df = merge_df[['city', 'district', 'total_floor', 'room', 'health']].copy()
other_df.drop_duplicates(inplace=True)
other_df.reset_index(drop=True, inplace=True)
other_df['num'] = range(len(other_df))
other_df.to_csv('3_merge_3.csv', index=False, na_rep='NULL')
print(other_df)

      city district total_floor room health    num
0      臺北市      文山區         十六層    2      2      0
1      臺北市      文山區        二十九層    3      2      1
2      臺北市      文山區         十六層    3      2      2
3      臺北市      中正區         十一層    1      1      3
4      臺北市      中正區         十一層    2      1      4
...    ...      ...         ...  ...    ...    ...
43530  澎湖縣      馬公市          一層    6      2  43530
43531  澎湖縣      馬公市          二層    5      5  43531
43532  澎湖縣      馬公市          四層    2      2  43532
43533  澎湖縣      馬公市          十層   16     16  43533
43534  澎湖縣      白沙鄉          二層    9      5  43534

[43535 rows x 6 columns]


In [11]:
merge_df['num1'] = -1
merge_df['num2'] = -1
merge_df['num3'] = -1
def add_num(row):
#     ['trade_target', 'area_use','compartment', 'manage']
#     tmp_df = land_df[(land_df['trade_target'] == row['trade_target']) & \
#               (land_df['area_use'] == row['area_use']) &\
#               (land_df['compartment'] == row['compartment']) &\
#               (land_df['manage'] == row['manage'])]
#     if(len(tmp_df)!=1):
#         print("error on num1")
#     row['num1'] = int(tmp_df['num'])
    
#     ['nonmetro_d', 'nonmetro_u', 'state', 'purpose', 'materials', 'hall', 'berth_type']
#     tmp_df = struct_df[(struct_df['nonmetro_d'] == row['nonmetro_d']) & \
#               (struct_df['nonmetro_u'] == row['nonmetro_u']) &\
#               (struct_df['state'] == row['state']) &\
#               (struct_df['purpose'] == row['purpose']) &\
#               (struct_df['materials'] == row['materials']) &\
#               (struct_df['hall'] == row['hall'])&\
#                 (struct_df['berth_type'] == row['berth_type'])]
#     if(len(tmp_df)!=1):
#         print("error on num2")
#     row['num2'] = int(tmp_df['num'])
    
#     ['city', 'district', 'total_floor', 'room', 'health']
#     tmp_df = other_df[(other_df['city'] == row['city']) & \
#               (other_df['district'] == row['district']) &\
#               (other_df['total_floor'] == row['total_floor']) &\
#               (other_df['room'] == row['room']) &\
#               (other_df['health'] == row['health'])]
#     if(len(tmp_df)!=1):
#         print("error on num3")
#     row['num3'] = int(tmp_df['num'])
    return row

last_df = merge_df

In [14]:
small_merge = last_df.drop(['trade_target', 'area_use','compartment', 'manage'\
                            ,'nonmetro_d', 'nonmetro_u', 'state', 'purpose', 'materials', 'hall', 'berth_type'\
                            ,'city', 'district', 'total_floor', 'room', 'health', 'yearmonth'], axis=1)
small_merge.to_csv('3_merge.csv', index=False, na_rep='NULL')
print(small_merge.shape)
print(small_merge.columns)

(724391, 16)
Index(['address', 'land_area', 'trade_date', 'target_detail', 'trade_floor',
       'building_date', 'building_area', 'price', 'unit_price', 'berth_area',
       'berth_price', 'note', 'trade_id', 'num1', 'num2', 'num3'],
      dtype='object')


In [13]:
tmp_df = pd.DataFrame([ x[:-4] for x in merge_df['trade_date']])
tmp_df[0].value_counts()

108    300667
107    298939
106     92593
105     11071
104      7068
103      4598
109      4214
102      2948
101      2019
100        81
99         61
96         20
98         19
97          9
92          8
80          7
93          6
95          6
91          5
18          4
84          4
86          4
70          4
94          3
78          3
65          3
66          3
62          2
64          2
87          2
88          2
85          2
56          1
71          1
77          1
79          1
75          1
89          1
81          1
69          1
15          1
14          1
82          1
17          1
76          1
63          1
Name: 0, dtype: int64